# Capstone Project 1 - Data Wrangling

Background:
- Data consists of information on all commercial U.S. domestic flights during Jan, Aug, Nov, and Dec of 2016

Procedure
1. Initialization
  - import and view raw data
2. Improve readability
  - reformat headers and reindex
  - address missing values
3. Evaluate remaining data
  - separate into dataframes based on component of flight information
  - address each component and reshape if necessary
  - join into one final Flights dataframe
4. Create final Links dataframes
  - link = unique origin to destination pair
  - utilize top 100 most traveled links as they contain high frequency of flights
  - create links dataframe consisting of median delay times per hour per link during sample period
5. Create networkx graph
6. Store finalized objects for further analysis

## 1. Initialization

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [2]:
#create dataframe from flight data csv
flights = pd.read_csv(r"C:\Users\mm183\Documents\Springboard\CP1\flight_data\flight_data.csv")

#explore data
print(flights.info())
print(flights.head())
print(flights.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1856061 entries, 0 to 1856060
Data columns (total 43 columns):
QUARTER                  int64
MONTH                    int64
DAY_OF_MONTH             int64
DAY_OF_WEEK              int64
FL_DATE                  object
UNIQUE_CARRIER           object
FL_NUM                   int64
ORIGIN_AIRPORT_ID        int64
ORIGIN_AIRPORT_SEQ_ID    int64
ORIGIN_CITY_MARKET_ID    int64
ORIGIN                   object
ORIGIN_CITY_NAME         object
ORIGIN_STATE_ABR         object
ORIGIN_STATE_NM          object
DEST_AIRPORT_ID          int64
DEST_AIRPORT_SEQ_ID      int64
DEST_CITY_MARKET_ID      int64
DEST                     object
DEST_CITY_NAME           object
DEST_STATE_ABR           object
DEST_STATE_NM            object
CRS_DEP_TIME             int64
DEP_TIME                 float64
DEP_DELAY                float64
DEP_DELAY_NEW            float64
WHEELS_ON                float64
TAXI_IN                  float64
CRS_ARR_TIME             int64

## 2. Surface cleaning and pre-processing
  - reformat headers and reindex
  - address missing values

In [3]:
#make column names lowercase
flights.columns = map(str.lower, flights.columns)

#view percentage of columns consisting of NaN
print(flights.isnull().mean())

quarter                  0.000000
month                    0.000000
day_of_month             0.000000
day_of_week              0.000000
fl_date                  0.000000
unique_carrier           0.000000
fl_num                   0.000000
origin_airport_id        0.000000
origin_airport_seq_id    0.000000
origin_city_market_id    0.000000
origin                   0.000000
origin_city_name         0.000000
origin_state_abr         0.000000
origin_state_nm          0.000000
dest_airport_id          0.000000
dest_airport_seq_id      0.000000
dest_city_market_id      0.000000
dest                     0.000000
dest_city_name           0.000000
dest_state_abr           0.000000
dest_state_nm            0.000000
crs_dep_time             0.000000
dep_time                 0.014290
dep_delay                0.014290
dep_delay_new            0.014290
wheels_on                0.015120
taxi_in                  0.015120
crs_arr_time             0.000000
arr_time                 0.015120
arr_delay     

In [4]:
#make lists of columns with high and low proportions of missing values 
high_per_nan = [col for col in flights if flights[col].isnull().mean() >= .5 ]
low_per_nan = [col for col in flights if 0 < flights[col].isnull().mean() < .5]

In [5]:
'''set aside columns containing high percentage of missing values
    - such columns contain circumstantial flight delay information for a small proportion of total flights
'''
#create dataframe containing high percentage NaN columns
hpn_cols = flights[high_per_nan]

#drop columns from flights dataframe
flights = flights.drop(columns=high_per_nan)

In [7]:
'''explore columns containing low percentage of missing values
    - create dataframe of columns with low percentage of NaN values
    - view descriptive statistics
    - view standard deviation of columns as percentage of mean to gauge variability
    - view percentage of total rows containing missing values
'''
lpn_cols = flights[low_per_nan]
print(lpn_cols.describe(include='all'))
print('coefficient of variation:','\n', lpn_cols.apply(lambda x: (x.std() / x.mean()) * 100))
print('percent of rows containing nan:', (len(flights[flights.isnull().any(axis=1)]) / len(flights)) * 100)

           dep_time     dep_delay  dep_delay_new     wheels_on       taxi_in  \
count  1.824403e+06  1.824403e+06   1.824403e+06  1.824403e+06  1.824403e+06   
mean   1.334294e+03  9.754709e+00   1.261746e+01  1.471571e+03  7.557781e+00   
std    5.004295e+02  4.153025e+01   4.050708e+01  5.267553e+02  6.028978e+00   
min    1.000000e+00 -6.000000e+01   0.000000e+00  1.000000e+00  1.000000e+00   
25%    9.200000e+02 -5.000000e+00   0.000000e+00  1.053000e+03  4.000000e+00   
50%    1.329000e+03 -2.000000e+00   0.000000e+00  1.510000e+03  6.000000e+00   
75%    1.741000e+03  7.000000e+00   7.000000e+00  1.914000e+03  9.000000e+00   
max    2.400000e+03  2.040000e+03   2.040000e+03  2.400000e+03  2.500000e+02   

           arr_time     arr_delay  arr_delay_new  crs_elapsed_time  \
count  1.824403e+06  1.824403e+06   1.824403e+06      1.824403e+06   
mean   1.476198e+03  4.206204e+00   1.259892e+01      1.464426e+02   
std    5.311709e+02  4.389959e+01   4.031154e+01      7.673842e+01   

In [6]:
'''set aside rows containing missing values from flights dataframe
    - due to high variability in critical columns (delay times) imputation of means would not be appropriate
    - rows constitute small percentage of total data
    - still possible at a later point to calculate missing values from remaining information
'''
#set aside dataframe of rows with null values
nan_rows = flights[flights.isnull().any(axis=1)]

#drop rows with null values from flights
flights = flights.drop(nan_rows.index)

#view percentage of remaining columns consisting of NaN 
print(flights.isnull().mean())

#view shape of resulting flights dataframe
print(flights.shape)

quarter                  0.0
month                    0.0
day_of_month             0.0
day_of_week              0.0
fl_date                  0.0
unique_carrier           0.0
fl_num                   0.0
origin_airport_id        0.0
origin_airport_seq_id    0.0
origin_city_market_id    0.0
origin                   0.0
origin_city_name         0.0
origin_state_abr         0.0
origin_state_nm          0.0
dest_airport_id          0.0
dest_airport_seq_id      0.0
dest_city_market_id      0.0
dest                     0.0
dest_city_name           0.0
dest_state_abr           0.0
dest_state_nm            0.0
crs_dep_time             0.0
dep_time                 0.0
dep_delay                0.0
dep_delay_new            0.0
wheels_on                0.0
taxi_in                  0.0
crs_arr_time             0.0
arr_time                 0.0
arr_delay                0.0
arr_delay_new            0.0
crs_elapsed_time         0.0
actual_elapsed_time      0.0
air_time                 0.0
flights       

## 3. Evaluate remaining data
  - separate into dataframes based on component of flight information
  - reshape if necessary
  - drop redundant columns

In [8]:
'''separate by columns based on component of flight information
    - date
    - identifiers (carrier, origin, destination)
    - status timestamps and other metrics
'''
#create dataframes for groups of columns
date_info = flights.loc[:, 'quarter':'fl_date']
id_info = flights.loc[:, 'unique_carrier':'dest_state_nm']
flight_metrics = flights.loc[:, 'crs_dep_time':'distance']

In [9]:
'''define function for creating dictionary with unique values in columns in given dataframe
    - key as column name
    - value as array of unique values in column 
    - sort array for readability
'''
def uv_dict(dataframe):
    unique_col_values = {col: array for (col, array) in [(col, dataframe[col].unique()) for col in dataframe]}
    for col, array in unique_col_values.items():
        array = array.sort()
    return unique_col_values

### Date

In [10]:
#view unique values in flight date columns
print(uv_dict(date_info))

#view columns pertaining to flight metrics
date_info.head()

{'quarter': array([1, 3, 4], dtype=int64), 'month': array([ 1,  8, 11, 12], dtype=int64), 'day_of_month': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype=int64), 'day_of_week': array([1, 2, 3, 4, 5, 6, 7], dtype=int64), 'fl_date': array(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
       '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
       '2016-01-09', '2016-01-10', '2016-01-11', '2016-01-12',
       '2016-01-13', '2016-01-14', '2016-01-15', '2016-01-16',
       '2016-01-17', '2016-01-18', '2016-01-19', '2016-01-20',
       '2016-01-21', '2016-01-22', '2016-01-23', '2016-01-24',
       '2016-01-25', '2016-01-26', '2016-01-27', '2016-01-28',
       '2016-01-29', '2016-01-30', '2016-01-31', '2016-08-01',
       '2016-08-02', '2016-08-03', '2016-08-04', '2016-08-05',
       '2016-08-06', '2016-08-07', '2016-08-08', '2016-08-09',
       '2016-08-10', '2016-08-11', '

,quarter,month,day_of_month,day_of_week,fl_date
0,1,1,3,7,2016-01-03
1,1,1,3,7,2016-01-03
2,1,1,3,7,2016-01-03
3,1,1,3,7,2016-01-03
4,1,1,3,7,2016-01-03


In [12]:
'''add columns containing day of week and month names
    -create dictionary matching numeric key to name 
    -map to each column to transform values to strings 
'''
#create dicts
dow_names = {1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7:'Sunday'}
month_names = {1:'January', 8:'August', 11:'November', 12:'December'}

#map dict to each column
date_info['Day_of_Week'] = date_info['day_of_week'].map(dow_names)
date_info['Month'] = date_info['month'].map(month_names)

In [11]:
'''create datetime index from flight departure time and flight date
    - reformat fl_date and dep_time strings and convert to datetime
    - add dt_index as column to date info dataframe
'''
#initialize empty list
fl_datetime = []

#loop through dep_time and fl_date columns to format then append flight date
for (i, v) in zip(flights.dep_time.astype(int).apply(str), flights.fl_date):
    if len(i) == 1:
        i = '000' + i 
    elif len(i) == 2:
        i = '00' + i 
    elif len(i) == 3:
        i = '0' + i
    elif i == '2400':
        i = '0000'
    else:
        i = i[:2] + i[2:]
    fl_datetime.append(v + i)

#add datetime index as column to flights dataframe
date_info['dt_index'] = pd.to_datetime(fl_datetime, format='%Y-%m-%d%H%M')

In [13]:
#add column containing categorical value for hour of day
date_info['hour_of_day'] = date_info['dt_index'].apply(lambda x: x.strftime('%H:00'))

#view final date info dataframe
date_info.head()

,quarter,month,day_of_month,day_of_week,fl_date,dt_index,Day_of_Week,Month,hour_of_day
0,1,1,3,7,2016-01-03,2016-01-03 15:24:00,Sunday,January,15:00
1,1,1,3,7,2016-01-03,2016-01-03 07:44:00,Sunday,January,07:00
2,1,1,3,7,2016-01-03,2016-01-03 18:58:00,Sunday,January,18:00
3,1,1,3,7,2016-01-03,2016-01-03 20:54:00,Sunday,January,20:00
4,1,1,3,7,2016-01-03,2016-01-03 12:52:00,Sunday,January,12:00


### Identification

****
#### Bureau of Transportation Statistics Definitions
(https://www.transtats.bts.gov/Fields.asp?Table_ID=292)
****

In [17]:
#create dictionary of unique values in id columns
id_uv = uv_dict(id_info)
print(id_uv)

#view columns pertaining to flight id
id_info.head()

{'unique_carrier': array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX',
       'WN'], dtype=object), 'fl_num': array([   1,    2,    3, ..., 7437, 7438, 7439], dtype=int64), 'origin_airport_id': array([10135, 10136, 10140, 10141, 10146, 10154, 10155, 10157, 10158,
       10165, 10170, 10185, 10208, 10245, 10257, 10279, 10299, 10333,
       10372, 10397, 10408, 10423, 10431, 10434, 10469, 10529, 10551,
       10561, 10577, 10581, 10599, 10620, 10627, 10631, 10666, 10685,
       10693, 10713, 10721, 10728, 10731, 10732, 10739, 10747, 10754,
       10779, 10781, 10785, 10792, 10800, 10821, 10849, 10868, 10874,
       10918, 10926, 10980, 10990, 10994, 11003, 11013, 11042, 11049,
       11057, 11066, 11076, 11097, 11109, 11122, 11140, 11146, 11150,
       11193, 11203, 11252, 11259, 11267, 11278, 11292, 11298, 11308,
       11336, 11337, 11413, 11423, 11433, 11447, 11471, 11481, 11495,
       11503, 11525, 11537, 11540, 11577, 11587, 11603, 11612, 11617,
       11618, 1

,unique_carrier,fl_num,origin_airport_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_nm,dest_airport_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_nm,link
0,F9,694,11292,30325,DEN,Denver,CO,Colorado,11003,31003,CID,Cedar Rapids/Iowa City,IA,Iowa,DEN-CID
1,F9,809,14027,34027,PBI,West Palm Beach/Palm Beach,FL,Florida,11292,30325,DEN,Denver,CO,Colorado,PBI-DEN
2,F9,907,15356,35356,TTN,Trenton,NJ,New Jersey,14492,34492,RDU,Raleigh/Durham,NC,North Carolina,TTN-RDU
3,F9,908,14492,34492,RDU,Raleigh/Durham,NC,North Carolina,15356,35356,TTN,Trenton,NJ,New Jersey,RDU-TTN
4,F9,915,15356,35356,TTN,Trenton,NJ,New Jersey,13930,30977,ORD,Chicago,IL,Illinois,TTN-ORD


In [15]:
"""remove state abbreviation suffix from city_name values
    - already present in state_abr columns
    - create function to remove abbreviations from list of columns in a dataframe
"""
def remove_abbr(dataframe, lst_of_columns):
    for col in lst_of_columns:
        dataframe[col] = dataframe[col].str[:-4]

#call function on id info for origin and destination city name columns 
remove_abbr(id_info,['origin_city_name','dest_city_name'])

In [14]:
#drop origin_airport_seq_id and dest_airport_seq_id
id_info.drop(['origin_airport_seq_id','dest_airport_seq_id'], 1, inplace=True)

In [16]:
'''create link column from origin and destination name columns
    - link = unique origin to destination pair
'''
id_info['link'] = id_info['origin'] + '-' + id_info['dest']

In [18]:
'''add unique carrier name column
    - create dict of unique_carrier codes to names
    - map to unique_carrier column to transform values to strings
    - carrier names were acquired from BTS statistics definitions
'''
unique_carrier_dict = {'F9':'Frontier Airlines Inc.','HA':'Hawaiian Airlines Inc.','DL':'Delta Air Lines Inc.',
                       'NK':'Spirit Air Lines','EV':'ExpressJet Airlines Inc.','OO':'SkyWest Airlines Inc.',
                       'UA':'United Air Lines Inc.','VX':'Virgin America','AA':'American Airlines Inc.',
                       'AS':'Alaska Airlines Inc.','B6':'JetBlue Airways','WN':'Southwest Airlines Co.'}

#map dict to unique_carrier column
id_info['unique_carrier_nm'] = id_info['unique_carrier'].map(unique_carrier_dict)

#view final dataframe on flight origin and destination
id_info.head()

,unique_carrier,fl_num,origin_airport_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_nm,dest_airport_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_nm,link,unique_carrier_nm
0,F9,694,11292,30325,DEN,Denver,CO,Colorado,11003,31003,CID,Cedar Rapids/Iowa City,IA,Iowa,DEN-CID,Frontier Airlines Inc.
1,F9,809,14027,34027,PBI,West Palm Beach/Palm Beach,FL,Florida,11292,30325,DEN,Denver,CO,Colorado,PBI-DEN,Frontier Airlines Inc.
2,F9,907,15356,35356,TTN,Trenton,NJ,New Jersey,14492,34492,RDU,Raleigh/Durham,NC,North Carolina,TTN-RDU,Frontier Airlines Inc.
3,F9,908,14492,34492,RDU,Raleigh/Durham,NC,North Carolina,15356,35356,TTN,Trenton,NJ,New Jersey,RDU-TTN,Frontier Airlines Inc.
4,F9,915,15356,35356,TTN,Trenton,NJ,New Jersey,13930,30977,ORD,Chicago,IL,Illinois,TTN-ORD,Frontier Airlines Inc.


### Metrics

****
#### Bureau of Transportation Statistics Definitions
(https://www.transtats.bts.gov/Fields.asp)
****

In [19]:
#view unique values in flight metrics columns
print(uv_dict(flight_metrics))

#view columns pertaining to flight metrics
flight_metrics.head()

{'crs_dep_time': array([   1,    2,    3, ..., 2357, 2358, 2359], dtype=int64), 'dep_time': array([1.000e+00, 2.000e+00, 3.000e+00, ..., 2.358e+03, 2.359e+03,
       2.400e+03]), 'dep_delay': array([ -60.,  -58.,  -54., ..., 1663., 1964., 2040.]), 'dep_delay_new': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.663e+03, 1.964e+03,
       2.040e+03]), 'wheels_on': array([1.000e+00, 2.000e+00, 3.000e+00, ..., 2.358e+03, 2.359e+03,
       2.400e+03]), 'taxi_in': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  

,crs_dep_time,dep_time,dep_delay,dep_delay_new,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,arr_delay_new,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance
0,1525,1524.0,-1.0,0.0,1807.0,8.0,1820,1815.0,-5.0,0.0,115.0,111.0,87.0,1.0,692.0
1,700,744.0,44.0,44.0,940.0,8.0,929,948.0,19.0,19.0,269.0,244.0,224.0,1.0,1679.0
2,1845,1858.0,13.0,13.0,2006.0,7.0,2015,2013.0,-2.0,0.0,90.0,75.0,60.0,1.0,373.0
3,2055,2054.0,-1.0,0.0,2208.0,7.0,2220,2215.0,-5.0,0.0,85.0,81.0,57.0,1.0,373.0
4,1250,1252.0,2.0,2.0,1349.0,51.0,1420,1440.0,20.0,20.0,150.0,168.0,107.0,1.0,693.0


In [20]:
'''remove flights column from metrics dataframe
    - all rows contain only one flight i.e. column values are redundant
'''
flight_metrics = flight_metrics.drop(columns='flights')

In [21]:
'''rename delay columns for clarity
    - dep_delay and arr_delay to dep_deviation and arr_deviation
      values can also indicate instances of early flights
    - dep_delay_new and arr_delay_new to dep_delay and arr_delay
      they are exclusively indicative of flight delay   
'''
flight_metrics.rename(columns={'dep_delay':'dep_deviation', 'dep_delay_new':'dep_delay',
                               'arr_delay':'arr_deviation','arr_delay_new':'arr_delay'}, inplace=True)

#view resulting flight metrics dataframe
flight_metrics.head()

,crs_dep_time,dep_time,dep_deviation,dep_delay,wheels_on,taxi_in,crs_arr_time,arr_time,arr_deviation,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
0,1525,1524.0,-1.0,0.0,1807.0,8.0,1820,1815.0,-5.0,0.0,115.0,111.0,87.0,692.0
1,700,744.0,44.0,44.0,940.0,8.0,929,948.0,19.0,19.0,269.0,244.0,224.0,1679.0
2,1845,1858.0,13.0,13.0,2006.0,7.0,2015,2013.0,-2.0,0.0,90.0,75.0,60.0,373.0
3,2055,2054.0,-1.0,0.0,2208.0,7.0,2220,2215.0,-5.0,0.0,85.0,81.0,57.0,373.0
4,1250,1252.0,2.0,2.0,1349.0,51.0,1420,1440.0,20.0,20.0,150.0,168.0,107.0,693.0


In [22]:
'''create final flights dataframe
    - join edited content-specific dataframes
    - create multiindex from link and flight departure datetime index, dt_index
'''
flights = date_info.join(id_info).join(flight_metrics).sort_values(by=['link','fl_date','dep_time'])

#create multiIndexed dataframe from link and dt_index columns
flights = flights.set_index(['link', 'dt_index'],drop=False).sort_index()

#view final flights dataframe
print(flights.info())
flights.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1824403 entries, (ABE-ATL, 2016-01-01 07:00:00) to (YUM-PHX, 2016-12-31 19:15:00)
Data columns (total 39 columns):
quarter                  int64
month                    int64
day_of_month             int64
day_of_week              int64
fl_date                  object
dt_index                 datetime64[ns]
Day_of_Week              object
Month                    object
hour_of_day              object
unique_carrier           object
fl_num                   int64
origin_airport_id        int64
origin_city_market_id    int64
origin                   object
origin_city_name         object
origin_state_abr         object
origin_state_nm          object
dest_airport_id          int64
dest_city_market_id      int64
dest                     object
dest_city_name           object
dest_state_abr           object
dest_state_nm            object
link                     object
unique_carrier_nm        object
crs_dep_time             int64
dep_t

quarter  month  day_of_month  day_of_week  \
link    dt_index                                                         
ABE-ATL 2016-01-01 07:00:00        1      1             1            5   
        2016-01-01 16:09:00        1      1             1            5   
        2016-01-02 06:00:00        1      1             2            6   
        2016-01-02 16:12:00        1      1             2            6   
        2016-01-03 05:57:00        1      1             3            7   

                                fl_date            dt_index Day_of_Week  \
link    dt_index                                                          
ABE-ATL 2016-01-01 07:00:00  2016-01-01 2016-01-01 07:00:00      Friday   
        2016-01-01 16:09:00  2016-01-01 2016-01-01 16:09:00      Friday   
        2016-01-02 06:00:00  2016-01-02 2016-01-02 06:00:00    Saturday   
        2016-01-02 16:12:00  2016-01-02 2016-01-02 16:12:00    Saturday   
        2016-01-03 05:57:00  2016-01-03 2016-01-03 05:57:00      Sunday   

                               Month hour_of_day unique_carrier   ...     \
link    dt_index                                                  ...      
ABE-ATL 2016-01-01 07:00:00  January       07:00             EV   ...      
        2016-01-01 16:09:00  January       16:00             EV   ...      
        2016-01-02 06:00:00  January       06:00             EV   ...      
        2016-01-02 16:12:00  January       16:00             EV   ...      
        2016-01-03 05:57:00  January       05:00             EV   ...      

                             wheels_on  taxi_in  crs_arr_time arr_time  \
link    dt_index                                                         
ABE-ATL 2016-01-01 07:00:00      900.0      7.0           845    907.0   
        2016-01-01 16:09:00     1812.0      5.0          1828   1817.0   
        2016-01-02 06:00:00      753.0      6.0           812    759.0   
        2016-01-02 16:12:00     1800.0      7.0          1827   1807.0   
        2016-01-03 05:57:00      753.0      5.0           813    758.0   

                            arr_deviation arr_delay crs_elapsed_time  \
link    dt_index                                                       
ABE-ATL 2016-01-01 07:00:00          22.0      22.0            135.0   
        2016-01-01 16:09:00         -11.0       0.0            138.0   
        2016-01-02 06:00:00         -13.0       0.0            132.0   
        2016-01-02 16:12:00         -20.0       0.0            137.0   
        2016-01-03 05:57:00         -15.0       0.0            133.0   

                             actual_elapsed_time  air_time distance  
link    dt_index                                                     
ABE-ATL 2016-01-01 07:00:00                127.0     111.0    692.0  
        2016-01-01 16:09:00                128.0     115.0    692.0  
        2016-01-02 06:00:00                119.0     101.0    692.0  
        2016-01-02 16:12:00                115.0      98.0    692.0  
        2016-01-03 05:57:00                121.0     107.0    692.0  

[5 rows x 39 columns]

## 4. Create NetworkX DiGraph

In [23]:
#group flights dataframe by link and slice for columns pertaining to location information
link_summary = flights.set_index(['link']).groupby(level='link').mean()[['distance','crs_elapsed_time']]

#iterate over location_summary and list of location name columns to add label strings
nm_lst = ['origin','origin_city_name','origin_state_nm','dest','dest_city_name','dest_state_nm']
for i in link_summary.index:
    for nm in nm_lst:
        link_summary.loc[i, nm] = flights.loc[i, nm][0]

#add number of flights per link as flight_count in link_summary dataframe
for i, v in flights.link.value_counts().iteritems():
    link_summary.loc[i, 'flight_count'] = float(v) 
    
#round crs_elapsed_time and sort location summary by origin state name to improve readability
link_summary = link_summary.round({'crs_elapsed_time': 0}).sort_values(by='origin_state_nm')

#normalize flights per link and add as weights to link_summary dataframe
link_summary['weight'] = MinMaxScaler().fit_transform(link_summary.flight_count.values.reshape(-1, 1))

#create networkx graph 
G = nx.from_pandas_edgelist(link_summary, 'origin', 'dest', ['weight','flight_count','distance','crs_elapsed_time'],
                            create_using=nx.DiGraph())

In [24]:
#create series of Degree distribution i.e. neighbors per node
deg_dist = pd.Series(data=[len([nbr for nbr in G.neighbors(n)]) for n in G.nodes()],
                        index=[n for n in G.nodes()]).sort_values(ascending=False)

#create series of Degree centrality i.e. # of neighbors/all neigbors possible per node
deg_cent = pd.Series(nx.degree_centrality(G)).sort_values(ascending=False)

#create series of Betweenness centrality i.e. # of shortest paths through node/ all short paths
bet_cent = pd.Series(nx.betweenness_centrality(G)).sort_values(ascending=False)

#create series of total outbound flights per node for the sample period
total_flights = flights.origin.value_counts()

#create dataframe of node attributes
node_attr_df = pd.DataFrame(data={'Total Flights':total_flights,'Degree':deg_dist,'Degree Centrality':deg_cent,
                                  'Betweenness Centrality':bet_cent}, index=list(G.nodes()),
                            columns=['Total Flights','Degree','Degree Centrality',
                                     'Betweenness Centrality']).sort_values('Total Flights', ascending=False)

#create dict of node attributes
node_attr_dict = node_attr_df.to_dict(orient='index')

'''create dict of node location name attributes from link summary 
   - rename origin to node for clarity
'''
origin_city_state = link_summary[['origin','origin_city_name','origin_state_nm']].drop_duplicates()
origin_city_state.columns = ['node', 'city_name','state_name']
name_attr_dict = origin_city_state.set_index('node').to_dict(orient='index')

#add attributes to nodes
nx.set_node_attributes(G, node_attr_dict)
nx.set_node_attributes(G, name_attr_dict)

## 5. Create final Links dataframes
  - link = unique origin to destination pair
  - utilize top 300 most traveled links as they contain high frequency of flights
  - create links dataframe consisting of median delay times per hour per link during sample period

In [26]:
#create series of link counts
link_counts = flights['link'].value_counts()

#create top links dataframe from list of top 300 pairs
top_links = flights[flights.link.isin(link_counts[:300].index.tolist())]

In [27]:
'''create list of dataframes resampled by link and hour
    - maintains link-specific temporal information
    - loop through multiIndex for top 100 links
    - resample link-specific dataframes by hour and append to empty list
'''
links_hourly_lst = []
for n in top_links.index.get_level_values(0).unique():
    n_df = top_links.loc[n]
    n_hourly = n_df.resample('60T').median().fillna(method='pad')
    n_hourly['link'] = n
    links_hourly_lst.append(n_hourly)

#concatenate resampled dataframes in list
links_hourly_concat = pd.concat(links_hourly_lst)

In [28]:
'''create links dataframe 
    - slice for months in original sample
    - avoids having final links dataframe consist of mostly filled values
    - object columns not yet dummified
'''
#create list of months in original sample
sample_months = top_links.index.get_level_values(1).month.unique().tolist()

#populate empty list with slices of the concatenated links_hourly dataframe containing sample months
links_s_months_lst= []
for month in sample_months:
    links_nmonth = links_hourly_concat.loc[links_hourly_concat.index.month == month]
    links_s_months_lst.append(links_nmonth)

#concatenate list of sample month slices 
links = pd.concat(links_s_months_lst)

#add datetime index as a column
links['dt_index'] = links.index

#add links column to the create multiIndex
links = links.set_index(['link','dt_index'],drop=False)

In [29]:
#define function to change id columns to names
def all_ids_to_names(df_w_names, df_w_ids, dict_of_cols):
    
    #define function for creating a dictionary of id-key: name-values, then adding names column in desired dataframe
    def name_for_id(df_w_name, df_w_id, name_col, id_col):
        nm_id_series = df_w_name[[name_col, id_col]].drop_duplicates().set_index(id_col)  
        nm_id_dict = {c_id: nm for (c_id, nm) in zip(nm_id_series.index.values, nm_id_series[name_col])}
        df_w_id[name_col] = df_w_id[id_col].replace(nm_id_dict)
        
    #loop through dictionary of all id and name columns and add names to dataframe containing only ids 
    for i, n in dict_of_cols.items():
        name_for_id(df_w_names, df_w_ids, n, i)
        
#create dictionary of id-name columns
id_name_dict = {'origin_airport_id':'origin', 'dest_airport_id':'dest',
                'origin_city_market_id':'origin_city_name', 'dest_city_market_id':'dest_city_name'}

#add airport and city names to links dataframe
all_ids_to_names(flights, links, id_name_dict)

In [30]:
'''drop redundant columns
    - fl_num column since flights have been aggregated
    - id columns since names have been added
    - dt_index column as MulitiIndex has already been created
'''
links = links.drop(['fl_num','origin_airport_id','origin_city_market_id',
                    'dest_airport_id','dest_city_market_id','dt_index'], axis=1)

#view final links dataframe
print(links.info())
links.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 882815 entries, (ANC-SEA, 2016-01-01 00:00:00) to (TPA-ATL, 2016-12-31 19:00:00)
Data columns (total 23 columns):
quarter                882815 non-null float64
month                  882815 non-null float64
day_of_month           882815 non-null float64
day_of_week            882815 non-null float64
crs_dep_time           882815 non-null float64
dep_time               882815 non-null float64
dep_deviation          882815 non-null float64
dep_delay              882815 non-null float64
wheels_on              882815 non-null float64
taxi_in                882815 non-null float64
crs_arr_time           882815 non-null float64
arr_time               882815 non-null float64
arr_deviation          882815 non-null float64
arr_delay              882815 non-null float64
crs_elapsed_time       882815 non-null float64
actual_elapsed_time    882815 non-null float64
air_time               882815 non-null float64
distance               882815 non-nul

quarter  month  day_of_month  day_of_week  \
link    dt_index                                                         
ANC-SEA 2016-01-01 00:00:00      1.0    1.0           1.0          5.0   
        2016-01-01 01:00:00      1.0    1.0           1.0          5.0   
        2016-01-01 02:00:00      1.0    1.0           1.0          5.0   
        2016-01-01 03:00:00      1.0    1.0           1.0          5.0   
        2016-01-01 04:00:00      1.0    1.0           1.0          5.0   

                             crs_dep_time  dep_time  dep_deviation  dep_delay  \
link    dt_index                                                                
ANC-SEA 2016-01-01 00:00:00          45.0      38.5           -6.5        0.0   
        2016-01-01 01:00:00          45.0      38.5           -6.5        0.0   
        2016-01-01 02:00:00         250.0     249.0           -1.0        0.0   
        2016-01-01 03:00:00         250.0     249.0           -1.0        0.0   
        2016-01-01 04:00:00         500.0     455.0           -5.0        0.0   

                             wheels_on  taxi_in      ...        arr_delay  \
link    dt_index                                     ...                    
ANC-SEA 2016-01-01 00:00:00      491.0      4.0      ...              8.0   
        2016-01-01 01:00:00      491.0      4.0      ...              8.0   
        2016-01-01 02:00:00      720.0      7.0      ...             17.0   
        2016-01-01 03:00:00      720.0      7.0      ...             17.0   
        2016-01-01 04:00:00      926.0      5.0      ...              1.0   

                             crs_elapsed_time  actual_elapsed_time  air_time  \
link    dt_index                                                               
ANC-SEA 2016-01-01 00:00:00             207.5                216.5     190.5   
        2016-01-01 01:00:00             207.5                216.5     190.5   
        2016-01-01 02:00:00             200.0                218.0     194.0   
        2016-01-01 03:00:00             200.0                218.0     194.0   
        2016-01-01 04:00:00             210.0                216.0     193.0   

                             distance     link  origin  dest origin_city_name  \
link    dt_index                                                                
ANC-SEA 2016-01-01 00:00:00    1448.0  ANC-SEA     ANC   SEA        Anchorage   
        2016-01-01 01:00:00    1448.0  ANC-SEA     ANC   SEA        Anchorage   
        2016-01-01 02:00:00    1448.0  ANC-SEA     ANC   SEA        Anchorage   
        2016-01-01 03:00:00    1448.0  ANC-SEA     ANC   SEA        Anchorage   
        2016-01-01 04:00:00    1448.0  ANC-SEA     ANC   SEA        Anchorage   

                            dest_city_name  
link    dt_index                            
ANC-SEA 2016-01-01 00:00:00        Seattle  
        2016-01-01 01:00:00        Seattle  
        2016-01-01 02:00:00        Seattle  
        2016-01-01 03:00:00        Seattle  
        2016-01-01 04:00:00        Seattle  

[5 rows x 23 columns]

In [31]:
'''create final links_d dataframe 
    - dummify link, quarter, month, day_of_week, day_of_month columns
'''
links_d = pd.get_dummies(links, columns=['link','quarter','month','day_of_week','day_of_month',
                                         'origin','origin_city_name','dest','dest_city_name'])

## Define target variables

In [32]:
'''write function to create series consisting of delay values at a future time
    - loop through each link since time-frames vary per sample i.e. not every link had traffic all 24 hours
    - shift series of delay values by -N amount to get values N hours away from the datetime in each row
    - forward fill missing values to maintain delay propagation dynamic
'''
def in_nhours(df, col, n_hours, grouping_level):
    future_lst = []
    for i in df.index.get_level_values(grouping_level).unique().sort_values():
        future_lst.append(df.loc[i, col].shift(-n_hours).fillna(method='ffill'))
    future_values_array = pd.concat(future_lst).values
    return future_values_array

In [34]:
#add column of departure delay values 6 hours past the current datetime to links_d dataframe
links_d = links_d.assign(dd_in_6hrs = in_nhours(links_d, 'dep_delay', 6, 'link'))

In [35]:
'''add target column to links_d dataframe
    - derived from departure delay in 6hrs in this case
    - 1 if delayed (15min past scheduled time per Bureau of Transportation statistics)
    - 0 otherwise
'''
links_d['dd_binary'] = (links_d['dd_in_6hrs'] >= 15)*1

In [36]:
#separate list of feature names from target columns
features = links_d.columns[:-3]

In [37]:
#view final links_d dataframe
print(links_d.info())
print(links_d.columns)
links_d.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 882815 entries, (ANC-SEA, 2016-01-01 00:00:00) to (TPA-ATL, 2016-12-31 19:00:00)
Columns: 561 entries, crs_dep_time to dd_binary
dtypes: float64(15), int32(1), uint8(545)
memory usage: 566.6+ MB
None
Index(['crs_dep_time', 'dep_time', 'dep_deviation', 'dep_delay', 'wheels_on',
       'taxi_in', 'crs_arr_time', 'arr_time', 'arr_deviation', 'arr_delay',
       ...
       'dest_city_name_San Diego', 'dest_city_name_San Jose',
       'dest_city_name_San Juan', 'dest_city_name_Seattle',
       'dest_city_name_St. Louis', 'dest_city_name_Tampa',
       'dest_city_name_Washington',
       'dest_city_name_West Palm Beach/Palm Beach', 'dd_in_6hrs', 'dd_binary'],
      dtype='object', length=561)


crs_dep_time  dep_time  dep_deviation  dep_delay  \
link    dt_index                                                                
ANC-SEA 2016-01-01 00:00:00          45.0      38.5           -6.5        0.0   
        2016-01-01 01:00:00          45.0      38.5           -6.5        0.0   
        2016-01-01 02:00:00         250.0     249.0           -1.0        0.0   
        2016-01-01 03:00:00         250.0     249.0           -1.0        0.0   
        2016-01-01 04:00:00         500.0     455.0           -5.0        0.0   

                             wheels_on  taxi_in  crs_arr_time  arr_time  \
link    dt_index                                                          
ANC-SEA 2016-01-01 00:00:00      491.0      4.0         512.5     515.0   
        2016-01-01 01:00:00      491.0      4.0         512.5     515.0   
        2016-01-01 02:00:00      720.0      7.0         710.0     727.0   
        2016-01-01 03:00:00      720.0      7.0         710.0     727.0   
        2016-01-01 04:00:00      926.0      5.0         930.0     931.0   

                             arr_deviation  arr_delay    ...      \
link    dt_index                                         ...       
ANC-SEA 2016-01-01 00:00:00            2.5        8.0    ...       
        2016-01-01 01:00:00            2.5        8.0    ...       
        2016-01-01 02:00:00           17.0       17.0    ...       
        2016-01-01 03:00:00           17.0       17.0    ...       
        2016-01-01 04:00:00            1.0        1.0    ...       

                             dest_city_name_San Diego  \
link    dt_index                                        
ANC-SEA 2016-01-01 00:00:00                         0   
        2016-01-01 01:00:00                         0   
        2016-01-01 02:00:00                         0   
        2016-01-01 03:00:00                         0   
        2016-01-01 04:00:00                         0   

                             dest_city_name_San Jose  dest_city_name_San Juan  \
link    dt_index                                                                
ANC-SEA 2016-01-01 00:00:00                        0                        0   
        2016-01-01 01:00:00                        0                        0   
        2016-01-01 02:00:00                        0                        0   
        2016-01-01 03:00:00                        0                        0   
        2016-01-01 04:00:00                        0                        0   

                             dest_city_name_Seattle  dest_city_name_St. Louis  \
link    dt_index                                                                
ANC-SEA 2016-01-01 00:00:00                       1                         0   
        2016-01-01 01:00:00                       1                         0   
        2016-01-01 02:00:00                       1                         0   
        2016-01-01 03:00:00                       1                         0   
        2016-01-01 04:00:00                       1                         0   

                             dest_city_name_Tampa  dest_city_name_Washington  \
link    dt_index                                                               
ANC-SEA 2016-01-01 00:00:00                     0                          0   
        2016-01-01 01:00:00                     0                          0   
        2016-01-01 02:00:00                     0                          0   
        2016-01-01 03:00:00                     0                          0   
        2016-01-01 04:00:00                     0                          0   

                             dest_city_name_West Palm Beach/Palm Beach  \
link    dt_index                                                         
ANC-SEA 2016-01-01 00:00:00                                          0   
        2016-01-01 01:00:00                                          0   
        2016-01-01 02:00:00                                      

## Step 7: Store finalized objects for further analysis

In [38]:
%store features
%store flights
%store G
%store links
%store links_d

Stored 'features' (Index)
Stored 'flights' (DataFrame)
Stored 'G' (DiGraph)
Stored 'links' (DataFrame)
Stored 'links_d' (DataFrame)
